In [1]:
#toggle relevant models
# with open('../models_pkl/random_forest.pkl', 'rb') as fit_rf:
#     clf = pickle.load(fit_rf)

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

import dill as pickle

np.random.seed(33)

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [3]:
from tensorflow.keras.models import model_from_json
with open('../cnn1.json') as f:
    model = model_from_json(f.read())

In [4]:
#model.load_weights('../cnn1_weights.h5')

In [105]:
df_val = pd.read_csv('../data/extra_labels.csv')

In [106]:
def pick_ylabels(column):
    y_val = df_val[column].copy().astype('category').cat.codes
    return (y_val.values)

In [107]:
loaded2 = np.load('../data/bw_images.npz')
X_val = loaded2['c']

In [120]:
val_true = pick_ylabels('masterCategory')[:20]

In [121]:
X_val = X_val.reshape(-1,80,60,1)[:20]

In [122]:
# get the predictions for the test data
predicted_classes = model.predict_classes(X_val)

# get the indices to be plotted
correct = np.nonzero(predicted_classes==val_true)[0]
incorrect = np.nonzero(predicted_classes!=val_true)[0]

In [129]:
df_val.iloc[incorrect, :]

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
9,55315,Men,Apparel,Socks,Socks,White,Summer,2016.0,Casual,Smartoe Men Pack of 3 Socks
18,43973,Women,Personal Care,Fragrance,Perfume and Body Mist,White,Spring,2017.0,Casual,DKNY Women Fresh Blossom 50 ml Perfume


### Test on Non-Sample

In [130]:
# from PIL import Image
# from skimage import transform,io, color
# from os import listdir
# import numpy as np
def process_images(directory):
    from PIL import Image
    from skimage import transform,io
    from os import listdir
    import numpy as np
    upload_array = []
    pic_list = [f for f in listdir(directory) if f.endswith('.png')]

    for pic in pic_list:
        fn = r'{}/{}'.format(directory, pic)
        img = io.imread(fn)
        img = transform.rescale(img, 3.0 / 4.0, anti_aliasing=False)
        if img.shape[2]==4:
            img = color.rgba2rgb(img)
            
        img_new = transform.resize(img, (80,60,3))
        upload_array.append(img_new.ravel())

    all_uploads = np.stack(upload_array)
    bw_uploads = all_uploads.reshape(-1, 80, 60, 3).mean(3).reshape(-1, 80*60).astype('float16') 
#     all_uploads = all_uploads.reshape(-1,80,60,3)
#     bw_uploads = bw_uploads.reshape(-1,80,60,1)
    return (all_uploads, bw_uploads)

In [174]:
def predict_master_cat(upload_array):
    predicted_category = model.predict_classes(upload_array)
    probability = np.round(model.predict_proba(upload_array),3)
    arr = np.zeros([len(predicted_category), 2])
    for i in range(len(predicted_category)):
        arr[i,:] = [predicted_category[i], probability[i][predicted_category[i]]]
    return arr

In [175]:
__, bw_new = process_images('image_uploads/')

In [176]:
bw_new = bw_new.reshape(-1,80,60,1)

In [181]:
new_predictions = np.round(predict_master_cat(bw_new),3)

In [182]:
new_predictions

array([[3.   , 0.333],
       [1.   , 0.531],
       [0.   , 0.606],
       [0.   , 0.462],
       [1.   , 0.663],
       [0.   , 0.583],
       [0.   , 0.481],
       [0.   , 0.431],
       [1.   , 0.381],
       [0.   , 0.414],
       [1.   , 0.705],
       [1.   , 0.54 ],
       [0.   , 0.391]])

In [183]:
model.predict_proba(bw_new)

array([[0.31813052, 0.313621  , 0.03552151, 0.33272696],
       [0.26676315, 0.53090686, 0.04458736, 0.15774257],
       [0.6059551 , 0.06892884, 0.03680033, 0.28831568],
       [0.46165392, 0.2214139 , 0.07092374, 0.24600837],
       [0.17286062, 0.6626007 , 0.0379009 , 0.12663788],
       [0.5834737 , 0.10829832, 0.07973107, 0.22849703],
       [0.48141152, 0.18362886, 0.10379785, 0.23116179],
       [0.43066067, 0.11295901, 0.05216604, 0.40421435],
       [0.36286056, 0.381282  , 0.0509335 , 0.20492397],
       [0.41409013, 0.26693985, 0.09283835, 0.22613162],
       [0.16096799, 0.7049589 , 0.02038956, 0.11368355],
       [0.2625734 , 0.5404398 , 0.03343431, 0.16355257],
       [0.39136824, 0.2985324 , 0.09592933, 0.21417001]], dtype=float32)